# LAB 2 - Similarity

In [ ]:
import jellyfish
import pandas as pd

# \u00a7 275.0-2 expressions
true_values = [
    {"doc_id": "§ 275.0-2_P1", "id": 1, "expression": "A person may serve process, pleadings, or other papers on a non-resident investment adviser, or on a non-resident general partner or non-resident managing agent of an investment adviser by serving any or all of its appointed agents."},
    {"doc_id": "§ 275.0-2_P1", "id": 2, "A person may serve a non-resident investment adviser, non-resident general partner, or non-resident managing agent by furnishing the Commission with one copy of the process, pleadings, or papers, for each named party, and one additional copy for the Commission's records.",
    "If process, pleadings, or other papers are served on the Commission as described in this section, the Secretary of the Commission (Secretary) will promptly forward a copy to each named party by registered or certified mail at that party's last address filed with the Commission.",
    "If the Secretary certifies that the Commission was served with process, pleadings, or other papers pursuant to paragraph (a)(1) of this section and forwarded these documents to a named party pursuant to paragraph (a)(2) of this section, this certification constitutes evidence of service upon that party.",
    "Managing agent means any person, including a trustee, who directs or manages, or who participates in directing or managing, the affairs of any unincorporated organization or association other than a partnership.",
    "Non-resident means an individual who resides in any place not subject to the jurisdiction of the United States.",
    "Non-resident means a corporation that is incorporated in or that has its principal office and place of business in any place not subject to the jurisdiction of the United States.",
    "Non-resident means a partnership or other unincorporated organization or association that has its principal office and place of business in any place not subject to the jurisdiction of the United States.",
    "Principal office and place of business has the same meaning as in \u00a7 275.203A-3(c) of this chapter."
]

# List of suspect domains to check for potential impersonation
pred_values = [
    "A person may serve process, pleadings, or other papers on a non-resident investment adviser, or on a non-resident general partner or non-resident managing agent of an investment adviser by serving any or all of its appointed agents.",
    "A person may serve a non-resident investment adviser, non-resident general partner, or non-resident managing agent by furnishing the Commission with one copy of the process, pleadings, or papers, for each named party, and one additional copy for the Commission's records.",
    "If process, pleadings, or other papers are served on the Commission as described in this section, the Secretary of the Commission (Secretary) will promptly forward a copy to each named party by registered or certified mail at that party's last address filed with the Commission.",
    "If the Secretary certifies that the Commission was served with process, pleadings, or other papers pursuant to paragraph (a)(1) of this section and forwarded these documents to a named party pursuant to paragraph (a)(2) of this section, this certification constitutes evidence of service upon that party.",
    "Managing agent means any person, including a trustee, who directs or manages, or who participates in directing or managing, the affairs of any unincorporated organization or association other than a partnership.",
    "Non-resident means an individual who resides in any place not subject to the jurisdiction of the United States.",
    "Non-resident means a corporation that is incorporated in or that has its principal office and place of business in any place not subject to the jurisdiction of the United States.",
    "Non-resident means a partnership or other unincorporated organization or association that has its principal office and place of business in any place not subject to the jurisdiction of the United States.",
    "Principal office and place of business has the same meaning as in \u00a7 275.203A-3(c) of this chapter.",
]

# Function to check the similarity between true and pred expression
def check_for_expressions_mililarity(true_list, pred_list, threshold=0.7):
    results = []
    for pred in pred_list:
        best_match = None
        best_score = 0
        for true_item in true_list:
            # Calculate the similarity score using the Levenshtein distance
            score = jellyfish.levenshtein_distance(pred, true_item)
            similarity_score = 1 - (score / max(len(pred), len(true_item)))  # Normalizing to a similarity score
            
            if similarity_score > best_score:
                best_score = similarity_score
                best_match = true_item
        results.append((pred, best_match, best_score))
    return results

# Check for potential domain impersonations
similarity_results = check_for_expressions_mililarity(true_values, pred_values)

# Building dataframe
df_results = pd.DataFrame(similarity_results, columns=["pred_expression", "true_expression", "similarity_score"])

# Adding Color Coding to "Similarity Score" Column
def highlight_similarity(val):
    color = 'green' if val >= 0.9 else 'red'  
    return f'background-color: {color}'
styled_df = df_results.style.applymap(highlight_similarity, subset=["similarity_score"])
styled_df.set_table_attributes('style="width: 100%; border: 1px solid black;"')
styled_df.set_properties(**{'border': '1px solid black'})


# Display the final DataFrame
styled_df

compare the id fields from both pred_list and true_list, we can enhance your code to analyze how often the pred_id matches the true_id 

In [ ]:
import jellyfish
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

# Function to group expressions by doc_id
def group_expressions_by_doc_id(expressions_list, is_pred=False):
    doc_id_map = defaultdict(list)
    for item in expressions_list:
        doc_id = item['doc_id']
        if is_pred:
            # Include filename, id, and classification for pred_list items
            expression_info = {
                'id': item['id'],
                'expression': item['expression'],
                'classification_pred': item.get('classification', None),
                'filename_pred': item['filename']
            }
        else:
            # Include id and classification for true_list items
            expression_info = {
                'id': item['id'],
                'expression': item['expression'],
                'classification_true': item.get('classification', None)
            }
        doc_id_map[doc_id].append(expression_info)
    return doc_id_map

# Function to check the similarity between true and pred expressions grouped by doc_id
def check_for_expressions_similarity(true_list, pred_list, threshold=0.7):
    # Build mappings from doc_id to list of expressions
    true_expressions_by_doc_id = group_expressions_by_doc_id(true_list)
    pred_expressions_by_doc_id = group_expressions_by_doc_id(pred_list, is_pred=True)

    results = []
    all_doc_ids = set(true_expressions_by_doc_id.keys()).union(pred_expressions_by_doc_id.keys())

    for doc_id in all_doc_ids:
        true_expressions = true_expressions_by_doc_id.get(doc_id, [])
        pred_expressions = pred_expressions_by_doc_id.get(doc_id, [])

        for pred_item in pred_expressions:
            pred_expr = pred_item['expression']
            pred_id = pred_item['id']
            classification_pred = pred_item.get('classification_pred', None)
            filename_pred = pred_item['filename_pred']
            best_match = None
            best_score = 0
            best_true_id = None
            classification_true = None
            for true_item in true_expressions:
                true_expr = true_item['expression']
                true_id = true_item['id']
                # Calculate the similarity score using the Levenshtein distance
                score = jellyfish.levenshtein_distance(pred_expr, true_expr)
                similarity_score = 1 - (score / max(len(pred_expr), len(true_expr)))  # Normalize to a similarity score

                if similarity_score > best_score:
                    best_score = similarity_score
                    best_match = true_expr
                    best_true_id = true_id
                    classification_true = true_item.get('classification_true', None)
            results.append({
                "doc_id": doc_id,
                "pred_id": pred_id,
                "true_id": best_true_id,
                "classification_pred": classification_pred,
                "classification_true": classification_true,
                "filename_pred": filename_pred,
                "pred_expression": pred_expr,
                "true_expression": best_match,
                "similarity_score": best_score
            })
    return results

# Use the function with your data
similarity_results = check_for_expressions_similarity(true_elements, pred_elements)

# Build the dataframe
df_results = pd.DataFrame(similarity_results)

# Add a column to indicate whether the pred_id matches the true_id
df_results['id_match'] = df_results['pred_id'] == df_results['true_id']

# Calculate the number and percentage of matches
total_matches = df_results['id_match'].sum()
total_comparisons = len(df_results)
match_percentage = (total_matches / total_comparisons) * 100

print(f"Total Matches: {total_matches}")
print(f"Total Comparisons: {total_comparisons}")
print(f"Match Percentage: {match_percentage:.2f}%")

# Create a confusion matrix for ID matching
# We'll label matches as 'match' and mismatches as 'mismatch'
df_results['id_match_label'] = df_results['id_match'].map({True: 'match', False: 'mismatch'})

# Generate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Prepare the data
y_true = df_results['id_match_label']  # Since we're comparing IDs, y_true and y_pred are the same
y_pred = df_results['id_match_label']

# Compute confusion matrix
labels = ['match', 'mismatch']
cm = confusion_matrix(y_true, y_pred, labels=labels)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(6, 4))
disp.plot(ax=ax, cmap=plt.cm.Blues)
plt.title('ID Match Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Generate classification report
report = classification_report(y_true, y_pred, labels=labels, digits=4)
print("Classification Report for ID Matching:\n")
print(report)


In [ ]:
df_results

## Queries

similarity search

```sparql
PREFIX llm: <http://franz.com/ns/allegrograph/8.0.0/llm/> 
PREFIX fibo: <https://spec.edmcouncil.org/fibo/ontology/master/2024Q2/QuickFIBOProd#>
PREFIX fro-cfr: <http://cfr2sbvr.com/fro/cfr/Code_Federal_Regulations.ttl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

select ?uri ?score (xsd:decimal(?score)*100 as ?score_percent) ?originalText ?definition
FROM fibo:FIBO_Graph
{
  (?uri ?score ?originalText) llm:nearestNeighbor ("employee" "test1-fibo" 5 0.9) .

  ?term a owl:Class ;
    skos:definition ?definition ;
    ?o ?originalText .
}
ORDER BY DESC(?score)
```

fibo-terms-definition

```sparql
# This PREFIX causes the default graph of the dataset to include
# only triples that are not in a named graph.
# Otherwise, the default graph will include every triple.
PREFIX fibo: <https://spec.edmcouncil.org/fibo/ontology/master/2024Q2/QuickFIBOProd#>
PREFIX fro-cfr: <http://cfr2sbvr.com/fro/cfr/Code_Federal_Regulations.ttl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

# View quads
SELECT ?term ?definition
#FROM fro-cfr:Code_Federal_Regulations_Graph
FROM fibo:FIBO_Graph
WHERE { 
  ?term a owl:Class ;
  skos:definition ?definition .
}
```

Qde embeddings

```sparkl
# Definition
select (COUNT(?s) AS ?qty_emb)
where {
  ?s <http://franz.com/vdb/gen/embedding> ?o . 
}
```

## AllegroGraph Cloud

In [ ]:
import sys

# Add the src (modules) directory to the path
sys.path.insert(0, '../src')

import configuration.main as configuration  # noqa: E402configuration.main as configuration

config = configuration.load_config()

In [ ]:
config

#### Logging

In [ ]:
import logging
from logging.handlers import TimedRotatingFileHandler
import os

# Ensure the ../logs directory exists
# log_directory = os.path.join(os.getcwd(), config["DEFAULT_LOG_DIR"])
# os.makedirs(log_directory, exist_ok=True)

# # Path for the log file
# log_file_path = os.path.join(log_directory, 'application.log')

# # Set up TimedRotatingFileHandler to rotate logs every day
# file_handler = TimedRotatingFileHandler(
#     log_file_path, when="midnight", interval=1, backupCount=0  # Rotate every midnight, keep all backups
# )

# # Set the file handler's log format
# file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S'))

# Set up logging configuration
logging.basicConfig(
    level=config["LOG_LEVEL"],  # Set to the desired log level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Console log format
    datefmt='%Y-%m-%d %H:%M:%S',  # Custom date format
    handlers=[
        #file_handler,  # Log to the rotating file in ../logs
        logging.StreamHandler()  # Log to console
    ]
)

# Example logger
logger = logging.getLogger(__name__)

# Log a test message to verify
#logger.info("Logging is set up with daily rotation.")

In [ ]:
f'https://{config["ALLEGROGRAPH"]["HOST"]}'

#### Localhost

In [ ]:
from franz.openrdf.connect import ag_connect

REPO= "actors"
CATALOG= "root"
HOST= "ag1eawvuu0p3zv35.allegrograph.cloud" # for AllegroGraph cloud
PORT= 443 # for AllegroGraph cloud
USER= "admin" # for AllegroGraph cloud
PASSWORD= ""  # Replace with your password
HOME_DIR= "/home/adsantos/agraph-8.2.1"

In [ ]:
conn = ag_connect(repo=REPO, catalog=config["ALLEGROGRAPH"]["CATALOG"],
                host=config["ALLEGROGRAPH"]["HOST"], port=config["ALLEGROGRAPH"]["PORT"],
                protocol="http",
                user=config["ALLEGROGRAPH"]["USER"], password=config["ALLEGROGRAPH"]["PASSWORD"])


In [ ]:
conn.addData("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
""")
query_string = "SELECT ?s ?p ?o { ?s ?p ?o . } LIMIT 10"

In [ ]:
from franz.openrdf.query.query import QueryLanguage

tuple_query = conn.prepareTupleQuery(QueryLanguage.SPARQL, query_string)
result = tuple_query.evaluate()

In [ ]:
with result:
   for binding_set in result:
        s = binding_set.getValue("s")
        p = binding_set.getValue("p")
        o = binding_set.getValue("o")
        print("%s %s %s" % (s, p, o))

In [ ]:
conn.close()

In [ ]:
from franz.openrdf.sail.allegrographserver import AllegroGraphServer
from franz.openrdf.repository.repository import Repository

server = AllegroGraphServer(host=config["ALLEGROGRAPH"]["HOST"], port=config["ALLEGROGRAPH"]["PORT"],
                            user=config["ALLEGROGRAPH"]["USER"], password=config["ALLEGROGRAPH"]["PASSWORD"],
                            repo=config["ALLEGROGRAPH"]["REPO"])

print(f"Available catalogs: {server.listCatalogs()}")

catalog = server.openCatalog('root')

print(f"Available repositories: {catalog.listRepositories()}")

myRepository = catalog.getRepository("actors", Repository.ACCESS)

print(f"Available databases: {myRepository.getDatabaseName()}")

#### Cloud

Workaround SSL problem

Stunnel for SSL

In [ ]:
%%writefile agraph_stunnel.conf

[allegrograph_proxy]
client = yes
accept = 127.0.0.1:8443
connect = ag1eawvuu0p3zv35.allegrograph.cloud:443

Start tunnel

In [ ]:
# Start
import getpass
import os

password = getpass.getpass()
command = "sudo -S stunnel agraph_stunnel.conf" #can be any command but don't forget -S as it enables input from stdin
os.system('echo %s | %s' % (password, command)) # Start stunnel

With ag_connect

In [ ]:
conn_cloud = ag_connect(repo=REPO, catalog=CATALOG,
                host="localhost", port=8443,
                protocol="http",
                user=USER, password=PASSWORD)

In [ ]:
tuple_query = conn_cloud.prepareTupleQuery(QueryLanguage.SPARQL, query_string)
result = tuple_query.evaluate()

with result:
   for binding_set in result:
        s = binding_set.getValue("s")
        p = binding_set.getValue("p")
        o = binding_set.getValue("o")
        print("%s %s %s" % (s, p, o))

With server class

In [ ]:
from franz.openrdf.sail.allegrographserver import AllegroGraphServer
from franz.openrdf.repository.repository import Repository

server = AllegroGraphServer(host="localhost", port=8443,
                            user=USER, password=PASSWORD, 
                            protocol="http")

print(f"Available catalogs: {server.listCatalogs()}")

catalog = server.openCatalog('root')

print(f"Available repositories: {catalog.listRepositories()}")

myRepository = catalog.getRepository("actors", Repository.ACCESS)

print(f"Available databases: {myRepository.getDatabaseName()}")

Close connection

In [ ]:
conn_cloud.close()

Stop tunnel

In [ ]:
# stop
password = getpass.getpass()
command = "sudo -S kill $(ps aux | grep 'stunnel agraph_stunnel.conf' | awk '{print $2}')" 
os.system('echo %s | %s' % (password, command)) # Start stunnel

In [ ]:
# section_num = "§ 275.0-7"
# logger.info(get_section_from_kg(conn, section_num=section_num))

Print results formatted

In [ ]:
# conn.close()